In [1]:
# Basic imports and general setup
import os, sys, importlib
from pathlib import Path
import numpy as np
import pandas as pd

# Had some issues with the file paths just bc of where I ran the notebook from but fixed by finding the root
repo_root = Path.cwd()
for up in [repo_root, repo_root.parent, repo_root.parent.parent]:
    if (up / "src").exists():
        repo_root = up
        break
sys.path.insert(0, str(repo_root))

# Importing geo module for coordinate translation. My Lat/Lon/Alt to ENU is in this geo.
import src.geo as geo 
importlib.invalidate_caches()
geo = importlib.reload(geo)

# Create clean alias for the main function we'll use
latlonalt_to_enu = geo.latlonalt_to_enu

#Making my notebook look cool
print(f"Repo root: {repo_root}")
print(f"Data path exists: {(repo_root / 'data' / 'raw').exists()}")

Repo root: /Users/danielvillafuerte/weather-aware-trajectory-prediction
Data path exists: True


In [2]:
#First step: Loading and starting the preprocessing
# Load the raw data
DATA_PATH = repo_root / "data" / "raw" / "06_12_17.csv"
df = pd.read_csv(DATA_PATH)

print(f"Loaded {len(df):,} rows")

# Time conversion and callsign cleanup
df["time"] = pd.to_datetime(df["time"], unit="s", errors="coerce")
df["callsign"] = df.get("callsign", "").fillna("").str.strip()

# Drop rows without position and time (essential for trajectory). Also realized that there are two opts for Altitude: baroaltitude and geoaltitude
df = df.dropna(subset=["lat", "lon", "time"]).copy()

# Create altitude column with fallback strategy
# Prefer geoaltitude, fallback to baroaltitude, then 0. Can imagine doing it this way would cause any issues.
if "geoaltitude" in df.columns:
    df["alt"] = df["geoaltitude"]
    if "baroaltitude" in df.columns:
        df["alt"] = df["alt"].fillna(df["baroaltitude"])
elif "baroaltitude" in df.columns:
    df["alt"] = df["baroaltitude"]
else:
    df["alt"] = 0.0
df["alt"] = df["alt"].fillna(0.0)

print(f"After filtering: {len(df):,} rows with valid position data")
df.head()

Loaded 1,561,361 rows
After filtering: 1,109,800 rows with valid position data


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,alt
0,2017-06-12 00:00:00,4062d5,52.741870,-0.567287,172.119770,295.492795,-9.10336,EXS46B,False,False,False,3244.0,9212.58,9372.60,1.497226e+09,1.497226e+09,9372.60
1,2017-06-12 00:00:10,a9f4c6,38.733215,-90.131836,150.662723,83.530180,13.00480,3536,False,False,False,1734.0,1905.00,1981.20,1.497226e+09,1.497226e+09,1981.20
2,2017-06-12 00:00:10,a0b20e,35.164902,-104.443939,221.811287,268.405170,0.00000,FDX556,False,False,False,7677.0,11574.78,12146.28,1.497226e+09,1.497226e+09,12146.28
5,2017-06-12 00:00:10,85c970,35.337385,138.482724,203.732815,264.203031,0.00000,ANA791,False,False,False,2355.0,7924.80,8161.02,1.497226e+09,1.497226e+09,8161.02
6,2017-06-12 00:00:10,a0ce85,39.907278,-104.448242,130.236658,73.712637,8.12800,,False,False,False,625.0,6515.10,3154.68,1.497225e+09,1.497226e+09,3154.68


In [3]:
# Keep Essential Columns & start Flight Segmentation
# Keep only columns relevant for trajectory prediction
essential_cols = ['time', 'icao24', 'lat', 'lon', 'alt', 'callsign']
optional_cols = ['velocity', 'heading', 'vertrate']  # Keep for potential validation but not really super necessary, I guess we will see

keep_cols = essential_cols + [c for c in optional_cols if c in df.columns] #list compehension to keep the optional if exists
df = df[keep_cols].copy()

print(f"Kept {len(df.columns)} columns: {list(df.columns)}")

# Sort by aircraft ID and time for proper segmentation between different flights
df = df.sort_values(['icao24', 'time']).reset_index(drop=True)

# Calculate time gaps between consecutive points for same aircraft
df['gap_s'] = df.groupby('icao24')['time'].diff().dt.total_seconds().fillna(0)

# Create flight_id: new flight when aircraft changes OR gap > 10 minutes
df['flight_id'] = ((df['icao24'] != df['icao24'].shift()) | 
                   (df['gap_s'] > 600)).cumsum()

print(f"Identified {df['flight_id'].nunique()} unique flight segments")
print(f"Median gap between points: {df[df['gap_s'] > 0]['gap_s'].median():.1f} seconds")

Kept 9 columns: ['time', 'icao24', 'lat', 'lon', 'alt', 'callsign', 'velocity', 'heading', 'vertrate']
Identified 7070 unique flight segments
Median gap between points: 10.0 seconds


In [4]:
# Filtering for Continuous Flight Segments
def keep_consecutive_runs(df, min_len=60, max_gap_s=120):
    """
    Keep only continuous, valid segments within each flight:
      - valid = lat, lon, alt present
      - new run when invalid OR time gap > max_gap_s
      - keep runs with at least min_len consecutive valid samples
    """
    d = df.sort_values(["flight_id", "time"]).copy()
    d["valid"] = d[["lat", "lon", "alt"]].notna().all(axis=1)
    d["dt"] = d.groupby("flight_id")["time"].diff().dt.total_seconds().fillna(0)

    # Mark breaks: invalid data OR gap too large
    break_flag = (~d["valid"]) | (d["dt"] > max_gap_s)
    d["run_id"] = break_flag.groupby(d["flight_id"]).cumsum()

    # Keep only valid points
    d_valid = d[d["valid"]].copy()
    
    # Count consecutive valid points per run
    run_sizes = d_valid.groupby(["flight_id", "run_id"]).size()
    keeps = run_sizes[run_sizes >= min_len].reset_index()[["flight_id", "run_id"]]
    
    # Filter to keep only long-enough runs
    out = d_valid.merge(keeps, on=["flight_id", "run_id"], how="inner")
    return out.drop(columns=["valid"])

# Apply filtering
df_runs = keep_consecutive_runs(df, min_len=60, max_gap_s=120).copy()
df_runs = df_runs.sort_values(["flight_id", "run_id", "time"]).reset_index(drop=True)

print(f"Kept {df_runs.groupby(['flight_id', 'run_id']).ngroups} continuous runs")
print(f"Total rows: {len(df_runs):,}")
print(f"Average run length: {df_runs.groupby(['flight_id', 'run_id']).size().mean():.1f} points")

Kept 5492 continuous runs
Total rows: 1,038,555
Average run length: 189.1 points


In [5]:
# Convert to Local ENU Coordinates finally using our geo
# Convert each run to ENU with origin at first point
runs = []
for (fid, rid), g in df_runs.groupby(["flight_id", "run_id"], sort=False):
    g = g.sort_values("time").copy()
    
    # Use first point as anchor/origin cause this is how ENU works 
    lat0, lon0, alt0 = g.iloc[0][["lat", "lon", "alt"]]
    
    # Convert entire trajectory to ENU
    E, N, U = latlonalt_to_enu(
        g["lat"].to_numpy(),
        g["lon"].to_numpy(),
        g["alt"].to_numpy(),
        float(lat0), float(lon0), float(alt0)
    )
    
    g["E"], g["N"], g["U"] = E, N, U
    runs.append(g)

df_runs = pd.concat(runs, ignore_index=True)
del runs  # Free memory

# Verify first point of each run is at origin
first_points = df_runs.groupby(["flight_id", "run_id"]).first()[["E", "N", "U"]]
print(f"Max E at origin: {first_points['E'].abs().max():.2e} m")
print(f"Max N at origin: {first_points['N'].abs().max():.2e} m") 
print(f"Max U at origin: {first_points['U'].abs().max():.2e} m")
print("\nSample ENU values (first 5 runs):")
print(df_runs.groupby(["flight_id", "run_id"])[["E", "N", "U"]].agg(['min', 'max']).head())

Max E at origin: 0.00e+00 m
Max N at origin: 0.00e+00 m
Max U at origin: 0.00e+00 m

Sample ENU values (first 5 runs):
                             E                           N                \
                           min           max           min           max   
flight_id run_id                                                           
1         0      -7.065782e-12  1.625722e-10 -2.183793e-10  8.848450e-10   
3         0       0.000000e+00  3.535112e+04  0.000000e+00  2.660262e+04   
          1       0.000000e+00  4.392011e+05  0.000000e+00  2.691027e+05   
4         0       0.000000e+00  4.690840e+05 -1.286783e+05  0.000000e+00   
7         0      -2.777987e+05  0.000000e+00 -3.867002e+05  0.000000e+00   

                             U            
                           min       max  
flight_id run_id                          
1         0      -12192.000000  0.000000  
3         0        -183.640069  5.794543  
          1      -20941.032764  0.000000  
4         0   

In [7]:
# Data Quality Check
# Calculate statistics for each run to identify potential issues
gb = df_runs.groupby(["flight_id", "run_id"], sort=False)

# First, let's add dt column if not present
df_runs["dt"] = gb["time"].diff().dt.total_seconds().fillna(10)

run_stats = gb.agg(
    duration_min=("time", lambda x: (x.max() - x.min()).total_seconds() / 60),
    n_points=("time", "count"),
    E_range=("E", lambda x: x.max() - x.min()),
    N_range=("N", lambda x: x.max() - x.min()),
    alt_change_km=("U", lambda x: (x.max() - x.min()) / 1000),
    cadence_s=("dt", "median")
).reset_index()

# Calculate approximate max distance from origin
run_stats["max_dist_km"] = np.sqrt(run_stats["E_range"]**2 + run_stats["N_range"]**2) / 1000

# Flag potentially problematic runs
run_stats["avg_speed_kmh"] = run_stats["max_dist_km"] / (run_stats["duration_min"] / 60)
run_stats["issues"] = ""
run_stats.loc[run_stats["avg_speed_kmh"] > 1200, "issues"] += "high_speed "
run_stats.loc[run_stats["avg_speed_kmh"] < 50, "issues"] += "too_slow "
run_stats.loc[run_stats["alt_change_km"] > 15, "issues"] += "excessive_alt_change "

print("Run statistics summary:")
print(run_stats[["duration_min", "max_dist_km", "alt_change_km", "avg_speed_kmh"]].describe().round(2))
print(f"\nRuns with issues: {(run_stats['issues'] != '').sum()} out of {len(run_stats)}")
print("\nSample problematic runs:")
problem_runs = run_stats[run_stats["issues"] != ""].head(10)
if len(problem_runs) > 0:
    print(problem_runs[["flight_id", "run_id", "duration_min", "avg_speed_kmh", "alt_change_km", "issues"]])

Run statistics summary:
       duration_min  max_dist_km  alt_change_km  avg_speed_kmh
count       5492.00      5492.00        5492.00        5492.00
mean          31.39       294.54          14.85         503.71
std           16.15       258.21          19.08         295.99
min            9.83         0.00           0.00           0.00
25%           17.67        86.19           2.82         267.31
50%           27.67       223.47           7.77         545.73
75%           43.50       439.89          19.16         733.84
max           59.67      2801.96         574.49        6226.57

Runs with issues: 2156 out of 5492

Sample problematic runs:
    flight_id  run_id  duration_min  avg_speed_kmh  alt_change_km  \
0           1       0     59.666667   1.122426e-12      12.192000   
2           3       1     31.833333   9.708432e+02      20.941033   
3           4       0     36.166667   8.069530e+02      18.545244   
4           7       0     47.666667   5.993369e+02      29.577920   
5 

In [8]:
# Compute Kinematics with Outlier Handling
# Calculate velocities and turn rates from ENU positions
gb = df_runs.groupby(["flight_id", "run_id"], sort=False)

# Time differences
df_runs["dt"] = gb["time"].diff().dt.total_seconds()
df_runs["dt"] = df_runs["dt"].replace(0, np.nan)  # Avoid division by zero

# Position differences
for c in ["E", "N", "U"]:
    df_runs[f"d{c}"] = gb[c].diff()

# Velocities (m/s)
df_runs["vE"] = df_runs["dE"] / df_runs["dt"]
df_runs["vN"] = df_runs["dN"] / df_runs["dt"]
df_runs["vU"] = df_runs["dU"] / df_runs["dt"]

# Cap unrealistic velocities (Mach 2 = ~680 m/s is a reasonable upper limit)
MAX_VELOCITY = 680  # m/s
for v in ["vE", "vN", "vU"]:
    df_runs.loc[df_runs[v].abs() > MAX_VELOCITY, v] = np.nan

# Speed and heading
df_runs["speed"] = np.sqrt(df_runs["vE"]**2 + df_runs["vN"]**2 + df_runs["vU"]**2)
df_runs["heading_rad"] = np.arctan2(df_runs["vE"], df_runs["vN"])

# Turn rate (using unwrapped heading to handle discontinuities)
df_runs["heading_unwrapped"] = gb["heading_rad"].transform(lambda x: np.unwrap(x.fillna(0)))
df_runs["turn_rate"] = gb["heading_unwrapped"].diff() / df_runs["dt"]

# Cap unrealistic turn rates (>0.5 rad/s = ~29 deg/s is extreme)
df_runs.loc[df_runs["turn_rate"].abs() > 0.5, "turn_rate"] = np.nan

print("Kinematics statistics after outlier handling:")
print(df_runs[["vE", "vN", "vU", "speed", "turn_rate"]].describe().round(2))
print(f"\nVelocity NaN fraction: {df_runs['speed'].isna().mean():.2%}")
print(f"Turn rate NaN fraction: {df_runs['turn_rate'].isna().mean():.2%}")

Kinematics statistics after outlier handling:
               vE          vN          vU       speed   turn_rate
count  1025788.00  1027937.00  1031965.00  1022955.00  1033063.00
mean         2.93       10.24       -5.93      142.11        0.00
std        148.03      112.37       19.04      118.59        0.06
min       -679.99     -679.96     -676.28        0.00       -0.31
25%        -87.52      -25.94      -10.67        0.00       -0.00
50%          0.00        0.00       -2.15      167.77        0.00
75%         89.47       65.08        0.00      232.55        0.00
max        679.89      679.77      677.35      961.38        0.31

Velocity NaN fraction: 1.50%
Turn rate NaN fraction: 0.53%


In [91]:
# Quality summaries (same helpers you wrote)
def enu_counts(df: pd.DataFrame, eps: float = 1e-6):
    total = len(df)
    out = {"total_rows": total}
    for c in ("E","N","U"):
        mask = df[c].notna() & (df[c].abs() > eps)
        cnt = int(mask.sum())
        out[f"{c}_nonzero_nonNaN"] = cnt
        out[f"{c}_pct"] = (cnt/total*100.0) if total else 0.0
    return pd.Series(out)

def enu_row_summary(df: pd.DataFrame, eps: float = 1e-6, ignore_first_per_flight: bool = True):
    needed = {"E","N","U"}
    if not needed.issubset(df.columns):
        raise KeyError("DataFrame must have E, N, U columns.")
    valid = df[["E","N","U"]].notna().all(axis=1)
    if ignore_first_per_flight and "flight_id" in df.columns:
        first_idx = df.groupby("flight_id").head(1).index
        valid_ex = valid.copy(); valid_ex.loc[first_idx] = False
    else:
        valid_ex = valid
    any_nz = valid_ex & (df[["E","N","U"]].abs() > eps).any(axis=1)
    all_nz = valid_ex & (df[["E","N","U"]].abs() > eps).all(axis=1)
    total = len(df); tot_valid = int(valid.sum())
    return pd.Series({
        "total_rows": total,
        "valid_rows": tot_valid,
        "valid_pct": (tot_valid/total*100.0) if total else 0.0,
        "any_nonzero_rows": int(any_nz.sum()),
        "any_nonzero_pct_of_valid": (int(any_nz.sum())/max(tot_valid,1)*100.0),
        "all_nonzero_rows": int(all_nz.sum()),
        "all_nonzero_pct_of_valid": (int(all_nz.sum())/max(tot_valid,1)*100.0),
        "ignored_first_rows": int(valid.sum() - valid_ex.sum()) if ignore_first_per_flight and "flight_id" in df.columns else 0
    })

print(enu_counts(df_runs, eps=1e-6))
print(enu_row_summary(df_runs, eps=1e-6, ignore_first_per_flight=True))


total_rows          1.038555e+06
E_nonzero_nonNaN    9.981860e+05
E_pct               9.611296e+01
N_nonzero_nonNaN    9.980340e+05
N_pct               9.609833e+01
U_nonzero_nonNaN    1.008920e+06
U_pct               9.714652e+01
dtype: float64
total_rows                  1.038555e+06
valid_rows                  1.038555e+06
valid_pct                   1.000000e+02
any_nonzero_rows            1.016795e+06
any_nonzero_pct_of_valid    9.790478e+01
all_nonzero_rows            9.909490e+05
all_nonzero_pct_of_valid    9.541613e+01
ignored_first_rows          5.335000e+03
dtype: float64


In [10]:
# Cell 8: Save Processed Data to Parquet
from pathlib import Path

# Create output directory
out_path = repo_root / "data" / "processed" / "flights_nativecadence_enu_kinematics.parquet"
out_path.parent.mkdir(parents=True, exist_ok=True)

# Ensure no Period dtypes that can cause issues with parquet
for c in df_runs.columns:
    if pd.api.types.is_period_dtype(df_runs[c]):
        df_runs[c] = df_runs[c].astype(str)

# Try saving with pyarrow first, fallback to fastparquet
saved = False

try:
    df_runs.to_parquet(out_path, index=False, engine="pyarrow", compression="snappy")
    print(f"✓ Saved with pyarrow → {out_path}")
    saved = True
except Exception as e:
    print(f"pyarrow failed: {e}")
    
if not saved:
    try:
        df_runs.to_parquet(out_path, index=False, engine="fastparquet", compression="snappy")
        print(f"✓ Saved with fastparquet → {out_path}")
        saved = True
    except Exception as e:
        print(f"fastparquet failed: {e}")
        # Last resort: save as CSV
        csv_path = out_path.with_suffix('.csv')
        df_runs.to_csv(csv_path, index=False)
        print(f"✓ Saved as CSV fallback → {csv_path}")

# Summary of what we saved
print(f"\nSaved data summary:")
print(f"- Total rows: {len(df_runs):,}")
print(f"- Unique flights: {df_runs['flight_id'].nunique():,}")
print(f"- Continuous runs: {df_runs.groupby(['flight_id', 'run_id']).ngroups:,}")

/var/folders/4j/b4y6wwhd0m18s4zqjd6p25mc0000gn/T/ipykernel_83205/1774761264.py:10: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df_runs[c]):


✓ Saved with pyarrow → /Users/danielvillafuerte/weather-aware-trajectory-prediction/data/processed/flights_nativecadence_enu_kinematics.parquet

Saved data summary:
- Total rows: 1,038,555
- Unique flights: 5,335
- Continuous runs: 5,492


================================================================================
DATA PREPARATION NOTEBOOK - OVERVIEW & OUTPUTS
================================================================================

STRATEGY:
---------
1. LOAD & CLEAN: Started with raw ADS-B data (06_12_17.csv) containing aircraft 
   position reports with many extraneous fields and potential quality issues.

2. FLIGHT SEGMENTATION: Grouped continuous tracks by aircraft (icao24) and time,
   creating new flights when gaps exceed 10 minutes. This handles cases where 
   the same aircraft appears multiple times in the dataset.

3. QUALITY FILTERING: Kept only continuous runs with:
   - At least 60 consecutive valid position reports
   - Time gaps < 120 seconds between points
   - Valid lat/lon/altitude throughout

4. COORDINATE TRANSFORMATION: Converted global lat/lon/alt to local East-North-Up
   (ENU) coordinates. Each flight segment uses its first point as the origin,
   making trajectories translation-invariant for ML models.

5. KINEMATICS COMPUTATION: Calculated velocities (vE, vN, vU) and turn rates
   from position data, with outlier capping at realistic aviation limits:
   - Max velocity: 680 m/s (Mach 2)
   - Max turn rate: 0.5 rad/s (29 deg/s)

OUTPUTS:
--------
- FILE: data/processed/flights_nativecadence_enu_kinematics.parquet
- SIZE: ~1M rows, 5,492 continuous flight segments
- COLUMNS: time, flight_id, run_id, lat, lon, alt, E, N, U, vE, vN, vU, 
          speed, heading_rad, turn_rate, dt, and metadata
- QUALITY: 96% of rows have valid ENU data, ready for trajectory prediction

KEY DECISIONS:
--------------
- Used 60-point minimum to ensure sufficient history for prediction
- ENU coordinates enable position-invariant learning
- Parquet format for efficient storage and fast loading
- Preserved native ~10s cadence (will resample in model-specific notebooks)

NEXT STEPS:
-----------
→ 02_ekf.ipynb: Physics-based Extended Kalman Filter baseline
→ 03_lstm.ipynb: Data-driven LSTM approach  
→ 04_compare.ipynb: Performance comparison with/without weather data
================================================================================